# Building a ReAct Agent from Scratch

This notebook demonstrates how to build a simple ReAct (Reasoning + Acting) agent using Ollama, corresponding to the SLM Hub [Agents Guide](https://slmhub.gitbook.io/slmhub/docs/learn/concepts/agents).

## 1. Setup
Install Ollama.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

In [ ]:
import subprocess
import time

# Start Ollama server in background
process = subprocess.Popen(["ollama", "serve"])
time.sleep(5)
print("Ollama started!")

## 2. Prepare Model
Pull a capable model like Phi-3 or Qwen2.5.

In [ ]:
!ollama pull phi3

## 3. Define the Agent
We Implement a simple ReAct loop loop that can use a calculator.

In [ ]:
import ollama
import re

# --- Tools ---
def calculator(expression):
    try:
        # Safety: restrict eval characters
        allowed = set("0123456789+-*/.() ")
        if not set(expression).issubset(allowed):
            return "Error: Invalid characters"
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

tools = {"calculator": calculator}

# --- Prompt ---
REACT_PROMPT = """
You are an intelligent agent. Solve the question step by step.
You have access to: [calculator]

Format:
Question: <input>
Thought: <reasoning>
Action: calculator
Action Input: <math_expression>
Observation: <result>
... repeat until answer ...
Final Answer: <the answer>

Question: {question}
"""

def run_agent(question):
    prompt = REACT_PROMPT.format(question=question)
    history = prompt
    print(f"Agent Start: {question}")

    for i in range(5):
        # 1. Think
        response = ollama.generate(model="phi3", prompt=history, options={"stop": ["Observation:"]})
        text = response['response']
        print(f"\n{text}")
        
        # 2. Check for Final Answer
        if "Final Answer:" in text:
            return text.split("Final Answer:")[1].strip()
            
        # 3. Check for Action
        match = re.search(r"Action: calculator\s*Action Input: (.*)", text, re.DOTALL)
        if match:
            expr = match.group(1).strip()
            result = tools["calculator"](expr)
            obs = f"Observation: {result}\n"
            print(obs.strip())
            history += text + "\n" + obs
        else:
            history += text

    return "Timed out"

# --- Run ---
ans = run_agent("What is 15 * 4 + 10?")
print(f"\n[Final Answer]: {ans}")